In [1]:
## exercise 11
## Team Chantalle_Tom
## date 22-01-2018

In [2]:
## set-up
from osgeo import ogr
from osgeo import osr
import os
import folium
import math


In [3]:
## create directories
try:
    os.stat("data")
except:
    os.mkdir("data") 

try:
    os.stat("output")
except:
    os.mkdir("output") 

In [4]:
## create two points from google maps (taken from the R vector tutorial)
wkt1 = "POINT (5.666473 51.985103)"
wkt2 = "POINT (5.679490 51.978954)"

In [5]:
## transform the projection to RD_New
source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(28992)

transform = osr.CoordinateTransformation(source,target)

point1 = ogr.CreateGeometryFromWkt(wkt1)
point2 = ogr.CreateGeometryFromWkt(wkt2)

point1.Transform(transform)
point2.Transform(transform)

0

In [6]:
## create the buffer
buffer1 = point1.Buffer(100,10)
buffer2 = point2.Buffer(100,10)
print (buffer1.Intersects(point1))
print (buffer2.Intersects(point2))

True
True


In [7]:
## create two shapefiles (for the buffer and the points)
driverName = "ESRI Shapefile"
drv = ogr.GetDriverByName( driverName )

spatialReference = osr.SpatialReference()
spatialReference.ImportFromEPSG(28992)

## create the shapefile for points
fn_points = "./data/points.shp"
lyn_points = "points"
ds_points = drv.CreateDataSource(fn_points)
layer_points = ds_points.CreateLayer(lyn_points, spatialReference, ogr.wkbPoint)

## create the shapefile for buffer
fn_buffer = "./data/buffer.shp"
lyn_buffer = "buffer"
ds_buffer = drv.CreateDataSource(fn_buffer)
layer_buffer = ds_buffer.CreateLayer(lyn_buffer, spatialReference, ogr.wkbPolygon)

In [8]:
## layer definition
layerDefinition1 = layer_points.GetLayerDefn()
layerDefinition2 = layer_buffer.GetLayerDefn()

## feature creation
feature1 = ogr.Feature(layerDefinition1)
feature2 = ogr.Feature(layerDefinition1)
feature3 = ogr.Feature(layerDefinition2)
feature4 = ogr.Feature(layerDefinition2)

feature1.SetGeometry(point1)
feature2.SetGeometry(point2)
feature3.SetGeometry(buffer1)
feature4.SetGeometry(buffer2)

layer_points.CreateFeature(feature1)
layer_points.CreateFeature(feature2)
layer_buffer.CreateFeature(feature3)
layer_buffer.CreateFeature(feature4)

print(layer_points.GetExtent())
print(layer_buffer.GetExtent())

(174184.37342619858, 175081.3233924724, 443434.6142300105, 444115.21165056876)
(174084.37342619858, 175181.3233924724, 443334.6142300105, 444215.21165056876)


In [9]:
## close the connection
ds_points.Destroy()
ds_buffer.Destroy()

In [10]:
## change shapefile extension to Geojson
!ogr2ogr -f GeoJSON -t_srs crs:84 data/points.geojson data/points.shp
!ogr2ogr -f GeoJSON -t_srs crs:84 data/buffer.geojson data/buffer.shp

In [11]:
## create a html with leaflet
pointsGeo = os.path.join("data/points.geojson")
bufferGeo = os.path.join("data/buffer.geojson")
map_total = folium.Map(location=[51.983095, 5.671786],tiles='OpenStreetMap', zoom_start=15)
map_total.choropleth(geo_data = pointsGeo)
map_total.choropleth(geo_data = bufferGeo)
map_total.save('output/points_buffer.html')

In [12]:
## calculate the distance with the haversine formula

coor1 = [5.666473, 51.985103]
coor2 = [5.679490, 51.978954]
lon1 = math.radians(coor1[0])
lon2 = math.radians(coor2[0])
lat1 = math.radians(coor1[1])
lat2 = math.radians(coor2[1])
dlon = lon2 - lon1 
dlat = lat2 - lat1 
a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
c = 2 * math.asin(math.sqrt(a)) 

## Radius of earth in kilometers is 6371
km = 6371* c
print (km)

1.1234926399935683
